In [1]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
cppinclude("ogdf/basic/graph_generators/deterministic.h")
cppinclude("ogdf/energybased/FMMMLayout.h")


G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.gridGraph(G, 3, 3, True,True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()
    GA.x[n] = (n.index() %3) * 50 
    GA.y[n] = (n.index()//3) * 50
    
#or adj in G.nodes[0].adjEntries:
#   print(adj.twinNode().index())
#   if adj.twinNode().index() == 9:
#       e = adj.theEdge()
#       GA.bends[e].pushBack(ogdf.DPoint(-100,-100)) 
    
#SL = ogdf.SugiyamaLayout()
#SL=ogdf.FMMMLayout();
 
#SL.useHighLevelOptions(True);
#SL.unitEdgeLength(15.0);
#SL.newInitialPlacement(True);
#SL.qualityVersusSpeed(ogdf.FMMMOptions.QualityVsSpeed.GorgeousAndEfficient);
#SL.call(GA)
ogdf.GraphIO.write(GA, "sugiyama-simple.graphml")

GA

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [2]:
%env OGDF_BUILD_DIR=~/OGDF-master/build-debug
from ogdf_python import ogdf, cppinclude
cppinclude("ogdf/basic/graph_generators/deterministic.h")

# 0 -- > x
# |
# v
#  y

def add_bends(line, *points):
    for point in points:
        line.pushBack(ogdf.DPoint(*point))

def find_wrap(GA, e):
    s,t = e.source(), e.target()
    wrap_h = GA.x[t] < GA.x[s]
    wrap_v = GA.y[t] < GA.y[s]
    if wrap_h and wrap_v:
        delta_x = abs(GA.x[t] - GA.x[s])
        delta_y = abs(GA.y[t] - GA.y[s])
        if delta_x > delta_y:
            wrap = "hv"
        else:
            wrap = "vh"
    elif wrap_h:
        wrap = "h"
    elif wrap_v:
        wrap = "v"
    else:
        wrap = ""
    return wrap

def apply_wrap(GA, box, e, wrap, margin=50):
    if not wrap:
        return
    
    s,t = e.source(), e.target()
    left = box.p1().m_x
    top = box.p1().m_y
    right = box.p2().m_x
    bottom = box.p2().m_y
    
    GA.bends[e].clear()
    if wrap[0] == "h":
        add_bends(GA.bends[e],
            (right + margin, GA.y[s]),
            (right + margin, top - margin),
            (left - margin, top - margin),
            (left - margin, GA.y[t]),
        )
    elif wrap[0] == "v":
        add_bends(GA.bends[e],
            (GA.x[s], bottom + margin),
            (left - margin, bottom + margin),
            (left - margin, top - margin),
            (GA.x[t], top - margin),
        )


def layout_edges(GA, box):
    wraps = ogdf.EdgeArray[str](GA.constGraph(), "")
    wraps_h = []
    wraps_v = []
    for e in GA.constGraph().edges:
        w = wraps[e] = find_wrap(GA, e)
        if w and w[0] == "h":
            wraps_h.append(e)
        elif w and w[0] == "v":
            wraps_v.append(e)
    wraps_h.sort(key=lambda e: GA.y[e.source()])
    wraps_v.sort(key=lambda e: GA.x[e.source()])
    
    for nr, e in enumerate(wraps_h + wraps_v):
        # GA.arrowType[e] = getattr(ogdf.EdgeArrow, "None")
        apply_wrap(GA, box, e, wraps[e], 50 + nr * 5)
        if wraps[e] in ["hv", "vh"]:
            GA.strokeColor[e] = ogdf.Color(0,255,0)
        elif wraps[e] == "h":
            GA.strokeColor[e] = ogdf.Color(255,0,0)
        elif wraps[e] == "v":
            GA.strokeColor[e] = ogdf.Color(0,0,255)
        
    return GA, wraps

def draw_bounding_box(G, GA, box):
    box_node1 = G.newNode()
    box_node2 = G.newNode()
    GA.x[box_node1] = box.p1().m_x - 25
    GA.y[box_node1] = box.p1().m_y - 25
    GA.x[box_node2] = box.p2().m_x + 25
    GA.y[box_node2] = box.p2().m_y + 25
    GA.width[box_node1] = 0
    GA.height[box_node1] = 0
    GA.width[box_node2] = 0
    GA.height[box_node2] = 0
    box_edge1 = G.newEdge(box_node1, box_node2)
    box_edge2 = G.newEdge(box_node1, box_node2)
    add_bends(GA.bends[box_edge1], (box.p1().m_x - 25, box.p2().m_y + 25))
    add_bends(GA.bends[box_edge2], (box.p2().m_x + 25, box.p1().m_y - 25))

env: OGDF_BUILD_DIR=~/OGDF-master/build-debug


In [3]:
width = 4
height = 4

G = ogdf.Graph()
ogdf.gridGraph(G, width, height, True, True)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.destroyAttributes(ogdf.GraphAttributes.nodeId)

for n in G.nodes:
    GA.label[n] = str(n.index())
    GA.x[n] = (n.index() % width) * 50 
    GA.y[n] = (n.index() // height) * 50

box = GA.boundingBox()
GA, wraps = layout_edges(GA, box)
# draw_bounding_box(G, GA, box)
GA

In [4]:
for e in G.edges:
    print(e.source().index(),e.target().index(),find_wrap(GA,e))
    

0 1 
1 2 
2 3 
3 0 h
0 4 
4 5 
1 5 
5 6 
2 6 
6 7 
3 7 
7 4 h
4 8 
8 9 
5 9 
9 10 
6 10 
10 11 
7 11 
11 8 h
8 12 
12 13 
9 13 
13 14 
10 14 
14 15 
11 15 
15 12 h
12 0 v
13 1 v
14 2 v
15 3 v


In [5]:
G.delNode(G.nodes[0])
G.delNode(G.nodes[1])
G.delNode(G.nodes[2])
G.delNode(G.nodes[4])
G.delNode(G.nodes[8])
G.delNode(G.nodes[12])
G.delNode(G.nodes[13])
G.delNode(G.nodes[11])
G.delNode(G.nodes[14])
G.delNode(G.nodes[15])


G.delEdge(G.searchEdge(G.nodes[5], G.nodes[6]))

# G.unsplit(G.nodes[15]) # replace deg 2 node by edge

G.newEdge(G.nodes[9],G.nodes[5])
G.newEdge(G.nodes[3],G.nodes[5])
G.newEdge(G.nodes[10],G.nodes[6])
G.newEdge(G.nodes[7],G.nodes[3])




layout_edges(GA, box)
GA

In [6]:
def adjList(nodes):
    
    adjList = {}
    
    for n in G.nodes:
        
        #adjList[GA.label[n]] = {}
        adjList[n] = {}
        
        for adj in n.adjEntries:
            
            e = adj.theEdge()
            wrap = find_wrap(GA, e)
            
            if n == e.source():
                
                if wrap == "h" or wrap == "hv":
                    position = 1
                    
                elif wrap == "v" or wrap == "vh":
                    position = 2
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 2
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 1
        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node lab
                adjList[n][position] = adj
                        
            else:
                
                if wrap == "h" or wrap == "hv":
                    position = 3
                    
                elif wrap == "v" or wrap == "vh":
                    position = 0
                    
                else:
                    if GA.x[n] == GA.x[adj.twinNode()]:
                        position = 0
                        
                    if GA.y[n] == GA.y[adj.twinNode()]:
                        position = 3
                        
                #adjList[GA.label[n]][position] = GA.label[adj.twinNode()]  #uncomment to use node labels
                adjList[n][position] = adj
                
    
    return adjList

result = adjList(G.nodes)

result

{<cppyy.gbl.ogdf.NodeElement object at 0x651bae0>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x64ef920>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x64f0130>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x64efe60>},
 <cppyy.gbl.ogdf.NodeElement object at 0x651bb60>: {2: <cppyy.gbl.ogdf.AdjElement object at 0x64efaa0>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x64ef800>,
  3: <cppyy.gbl.ogdf.AdjElement object at 0x64f0100>},
 <cppyy.gbl.ogdf.NodeElement object at 0x651bba0>: {1: <cppyy.gbl.ogdf.AdjElement object at 0x64ef8c0>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x64efb60>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x64efec0>},
 <cppyy.gbl.ogdf.NodeElement object at 0x651bbe0>: {3: <cppyy.gbl.ogdf.AdjElement object at 0x64ef8f0>,
  0: <cppyy.gbl.ogdf.AdjElement object at 0x64ef950>,
  2: <cppyy.gbl.ogdf.AdjElement object at 0x64efe90>},
 <cppyy.gbl.ogdf.NodeElement object at 0x651bc60>: {0: <cppyy.gbl.ogdf.AdjElement object at 0x64efad0>,
  1: <cppyy.gbl.ogdf.AdjElement object at 0x

In [7]:
def sorted_adjList(adjList):
    
    sorted_adjList = {}
    
    for n in adjList:
        
        sortedList = []
        
        for i in sorted (adjList[n]) :
            
            sortedList.append(adjList[n][i])
        
        sorted_adjList[n] = sortedList
    
    return sorted_adjList
    
    
sorted_adjList = sorted_adjList(result)
sorted_adjList

{<cppyy.gbl.ogdf.NodeElement object at 0x651bae0>: [<cppyy.gbl.ogdf.AdjElement object at 0x64efe60>,
 <cppyy.gbl.ogdf.NodeElement object at 0x651bb60>: [<cppyy.gbl.ogdf.AdjElement object at 0x64ef800>,
 <cppyy.gbl.ogdf.NodeElement object at 0x651bba0>: [<cppyy.gbl.ogdf.AdjElement object at 0x64efec0>,
 <cppyy.gbl.ogdf.NodeElement object at 0x651bbe0>: [<cppyy.gbl.ogdf.AdjElement object at 0x64ef950>,
 <cppyy.gbl.ogdf.NodeElement object at 0x651bc60>: [<cppyy.gbl.ogdf.AdjElement object at 0x64efad0>,
 <cppyy.gbl.ogdf.NodeElement object at 0x651bca0>: [<cppyy.gbl.ogdf.AdjElement object at 0x64efb90>,
  <cppyy.gbl.ogdf.AdjElement object at 0x64efb30>]}

In [8]:
def sortAdjs(node, order):
    l = ogdf.List["ogdf::adjEntry"]()
    for adj in order:
        l.pushBack(adj)
    node.adjEntries.sort(l)

node3 = list(sorted_adjList)[0]
node3_adjEntries = sorted_adjList[node3]
sortAdjs(node3, node3_adjEntries)
list(node3.adjEntries)


In [9]:
for n in sorted_adjList:
    
    node_adjEntries = sorted_adjList[n]
    sortAdjs(n, node_adjEntries)
    print(list(n.adjEntries))

   

[<cppyy.gbl.ogdf.AdjElement object at 0x64efe60>, <cppyy.gbl.ogdf.AdjElement object at 0x64f0130>, <cppyy.gbl.ogdf.AdjElement object at 0x64ef920>]
[<cppyy.gbl.ogdf.AdjElement object at 0x64ef800>, <cppyy.gbl.ogdf.AdjElement object at 0x64efaa0>, <cppyy.gbl.ogdf.AdjElement object at 0x64f0100>]
[<cppyy.gbl.ogdf.AdjElement object at 0x64efec0>, <cppyy.gbl.ogdf.AdjElement object at 0x64ef8c0>, <cppyy.gbl.ogdf.AdjElement object at 0x64efb60>]
[<cppyy.gbl.ogdf.AdjElement object at 0x64ef950>, <cppyy.gbl.ogdf.AdjElement object at 0x64efe90>, <cppyy.gbl.ogdf.AdjElement object at 0x64ef8f0>]
[<cppyy.gbl.ogdf.AdjElement object at 0x64efad0>, <cppyy.gbl.ogdf.AdjElement object at 0x64efb00>, <cppyy.gbl.ogdf.AdjElement object at 0x64ef830>]
[<cppyy.gbl.ogdf.AdjElement object at 0x64efb90>, <cppyy.gbl.ogdf.AdjElement object at 0x64efef0>, <cppyy.gbl.ogdf.AdjElement object at 0x64efb30>]


In [10]:
G.genus()

1

In [11]:
#for computing the dual

import cppyy
dualG = ogdf.Graph()
# maps G.adjEntry -> dualG.node (face)
dualFace = ogdf.AdjEntryArray["ogdf::node"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::NodeElement"))
# maps G.edge -> dualG.edge
dualEdge = ogdf.EdgeArray["ogdf::edge"](G, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))
# maps dualG.edge -> G.edge (== dualEdge^{-1})
primalEdge = ogdf.EdgeArray["ogdf::edge"](dualG, cppyy.bind_object(cppyy.nullptr, "ogdf::EdgeElement"))

for n in G.nodes:
    for adj in n.adjEntries:
        if dualFace[adj]:
            continue
        face = dualG.newNode()
        dualFace[adj] = face
        
        a = adj.faceCycleSucc()
        while a != adj:
            dualFace[a] = face
            a = a.faceCycleSucc()

for e in G.edges:
    de = dualG.newEdge(dualFace[e.adjSource()], dualFace[e.adjTarget()])
    dualEdge[e] = de
    primalEdge[de] = e

In [12]:
#for computing the spanning trees 

for e in G.edges:
    GA.strokeColor[e] = ogdf.Color(0,0,0)

# type of G edges: 0 (black) is in X, 1 (red) is edge of spanning cotree, 2 (green) is edge of primal ST
tree = ogdf.EdgeArray[int](G, 0)

found = ogdf.NodeArray[bool](dualG, False)
first_node = next(iter(G.nodes))
found[first_node] = True
todo = list(dualG.nodes[0].adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()]:
        continue

    tree[primalEdge[cur.theEdge()]] = 1
    GA.strokeColor[primalEdge[cur.theEdge()]] = ogdf.Color(255,0,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()]:
            todo.append(adj)
print(count, dualG.numberOfNodes())

found = ogdf.NodeArray[bool](G, False)
first_node = next(iter(G.nodes))
found[first_node] = True
todo = list(first_node.adjEntries)
count = 1
while len(todo) > 0:
    cur = todo.pop(-1)
    if found[cur.twinNode()] or tree[cur] != 0:
        continue

    tree[cur.theEdge()] = 2
    GA.strokeColor[cur.theEdge()] = ogdf.Color(0,255,0)
    found[cur.twinNode()] = True
    count += 1

    for adj in cur.twinNode().adjEntries:
        if not found[adj.twinNode()] and tree[adj] == 0:
            todo.append(adj)

print(count, G.numberOfNodes())

unlabeled = [e for e in G.edges if tree[e] == 0]
print(unlabeled)
GA


4 3
6 6
[<cppyy.gbl.ogdf.EdgeElement object at 0x6524258>, <cppyy.gbl.ogdf.EdgeElement object at 0x6524108>]


In [13]:
for e in G.edges:
    if tree[e] == 0:
        start_edge = e
        break
    

In [14]:
#start from any edge, and any of its node, check adjacent edges of the node, if already set, go to the next

import math
orientation = ogdf.EdgeArray[int](G,99)


def bendCoord(e):
    bends = [(b.m_x,b.m_y) for b in GA.bends[e]]
    return bends

def findAngle(node,x,y): #angle relative to the current node 
    refx = GA.x[node]
    refy = GA.y[node]
    rad = math.atan2(refy-y,x-refx)
    deg = math.degrees(rad)
    return deg

def findDirection(refNode,e1,e2): #turning left -1 /right 1 /straight 0
    
    if len(bendCoord(e1)) != 0:
                    
        if e1.target() == refNode:
                    
            bendpoint = bendCoord(e1)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
            
                    
            a1 = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            
            bendpoint = bendCoord(e1)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
           
                    
            a1 = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if e1.target() == refNode:
            nextNode = e1.source()
            a1 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
        else:
            nextNode = e1.target()
            a1 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
        
    if len(bendCoord(e2)) != 0:
               
        if e2.target() == refNode:
            bendpoint = bendCoord(e2)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            a2 = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            bendpoint = bendCoord(e2)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            a2 = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if e2.target() == refNode:
            nextNode = e2.source()
            a2 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
           
        else:
            nextNode = e2.target()
            a2 = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
            
    
    if a1 == 180: 
        if a2 == 90:
            direction = -1
        elif a2 == 0:
            direction = 0
        elif a2 == -90:
            direction = 1
            
    elif a1 == -90:
        if a2 == 90:
            direction = 0
        elif a2 == 0:
            direction = 1
        elif a2 == 180:
            direction = -1
            
    elif a1 == 90:
        if a2 == -90:
            direction = 0
        elif a2 == 0:
            direction = -1
        elif a2 == 180:
            direction = 1
            
    elif a1 == 0:
        if a2 == -90:
            direction = -1
        elif a2 == 90:
            direction = 1
        elif a2 == 180:
            direction = 0
    print(a1,a2)
    return direction

def defineOrientation(refEdge,refNode,refOrientation): #either vertical or horizontal 
    
    
    if len(bendCoord(refEdge)) != 0:
        if refEdge.target() == refNode:          
            bendpoint = bendCoord(refEdge)[-1]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
        else:
            bendpoint = bendCoord(refEdge)[0]
            bendpointx = bendpoint[0]
            bendpointy = bendpoint[1]
                    
            refAngle = findAngle(refNode,bendpointx,bendpointy)
                        
    else:
        if refEdge.target() == refNode:        
            refAngle = findAngle(refNode,GA.x[refEdge.source()],GA.y[refEdge.source()])
                        
        else:         
            refAngle = findAngle(refNode,GA.x[refEdge.target()],GA.y[refEdge.target()])
        
    
    
    for adj in refNode.adjEntries:
        e= adj.theEdge()
        nextNode = adj.twinNode()
        #print(refNode.index(),nextNode.index())
        if e!= refEdge:
            if orientation[e] == 0 or orientation [e] == 1: #to distinguish vertical 1 and horizontal 0
                #print("this edge is done")
                continue
                
            else:
                if len(bendCoord(e)) != 0:
                    #print("has bend")
                    if e.target() == refNode:
                    
                        bendpoint = bendCoord(e)[-1]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                    else:
                        bendpoint = bendCoord(e)[0]
                        bendpointx = bendpoint[0]
                        bendpointy = bendpoint[1]
                    
                        angle = findAngle(refNode,bendpointx,bendpointy)
                        
                else:
                    angle = findAngle(refNode,GA.x[nextNode],GA.y[nextNode])
                    
            
            if abs(angle)%180 == abs(refAngle)%180:
                orientation[e] = refOrientation
                    
            else: 
                orientation[e] = 1-refOrientation
                    
            
                    
            defineOrientation(e,nextNode,orientation[e])
    
    return
   
        
        
def findGenerators(start_edge, source, target, generator):
   
    for adj in target.adjEntries:
        
        if tree[adj.theEdge()] == 2 and adj.theEdge() != start_edge:
            cur_edge = adj.theEdge()
            next_node = adj.twinNode()
            generator.append(cur_edge)
        
            if next_node == source:
                return True, generator
            else:
                bool, generator = findGenerators(cur_edge, source, next_node,generator)
                if bool:
                    return True, generator
        
    generator.remove(start_edge)
    return False, generator
        
def checkCoeff(loop, variableList):
    if len(loop) > 0:
        sum = 0
        for e in loop:
            sum += variableList[e]
            
        if sum < 0:
            rhs = -1
        else:
            rhs = 1
            
    else:
        rhs = 0
        
    return rhs

In [15]:
defineOrientation(start_edge,start_edge.source(),0)
for e in G.edges:
    print(e.source().index(), e.target().index(), orientation[e])

6 7 0
3 7 1
5 9 1
9 10 0
6 10 1
9 5 1
3 5 0
10 6 1
7 3 1


In [16]:
#the black edges that must be included in each generator
starter_edges = [e for e in G.edges if tree[e] == 0]


start_edge = starter_edges[0]
betaLoop = [start_edge]
bool, betaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(), betaLoop)
print("beta loop")
for i, e in enumerate(betaLoop):
    print(betaLoop[i].source().index(),betaLoop[i].target().index())

print("alpha loop")
start_edge = starter_edges[1]
alphaLoop = [start_edge]
bool, alphaLoop = findGenerators(start_edge, start_edge.source(),start_edge.target(),alphaLoop)
for i, e in enumerate(alphaLoop):
    print(alphaLoop[i].source().index(),alphaLoop[i].target().index())

beta loop
3 5
5 9
9 10
6 10
6 7
3 7
alpha loop
10 6
6 10


In [17]:
for i, e in enumerate(alphaLoop):
    if i < len(alphaLoop)-1:
        print(e.source().index(),e.target().index(), findDirection(e.commonNode(alphaLoop[i+1]),alphaLoop[i],alphaLoop[i+1]))
    else:
        print(alphaLoop[i].source().index(),alphaLoop[i].target().index(), findDirection(e.commonNode(alphaLoop[0]),alphaLoop[i],alphaLoop[0]))
        
for i, e in enumerate(betaLoop):
    if i < len(betaLoop)-1:
        print(e.source().index(),e.target().index(), findDirection(e.commonNode(betaLoop[i+1]),betaLoop[i],betaLoop[i+1]))
    else:
        print(betaLoop[i].source().index(),betaLoop[i].target().index(), findDirection(e.commonNode(betaLoop[0]),betaLoop[i],betaLoop[0]))
        


-90.0 90.0
10 6 0
-90.0 90.0
6 10 0
180.0 -90.0
3 5 1
90.0 0.0
5 9 -1
180.0 90.0
9 10 -1
-90.0 0.0
6 10 1
180.0 90.0
6 7 -1
-90.0 0.0
3 7 1


In [18]:
from pulp import *
import pandas as pd
import numpy as np

#give an orientation to each edge
alphaLoopVertical = []
alphaLoopHorizontal = []
signs1 = {}
sign = 999
for i, e in enumerate(alphaLoop):
    
    if i < len(alphaLoop)-1:
        ei = alphaLoop[i]
        ei1 = alphaLoop[i+1]
        
        turn = findDirection(e.commonNode(ei1),e,ei1)
        if turn != 0 and turn != -1*sign:
            if sign == 999:
                sign = turn
                signs1[ei1] = sign
            else:
                sign = -1*sign
                signs1[ei1] = sign
        else:
            signs1[ei1] = 1
        
    else:
        ei = alphaLoop[i]
        ei1 = alphaLoop[0]
        
        turn = findDirection(e.commonNode(ei1),e,ei1)
        if turn != 0 and turn != -1*sign:
            if sign == 999:
                sign = turn
                signs1[ei1] = sign
            else:
                sign = -1*sign
                signs1[ei1] = sign   
        else:
            signs1[ei1] = 1
            
        
    if orientation[e] == 0:
        alphaLoopHorizontal.append(e)
    else:
        alphaLoopVertical.append(e)


-90.0 90.0
-90.0 90.0


In [19]:
betaLoopVertical = []
betaLoopHorizontal = []
signs2 = {}
flags = {}
prev_turn = 999
for i, e in enumerate(betaLoop):
    
    if i < len(betaLoop)-1:
        ei = betaLoop[i]
        ei1 = betaLoop[i+1]           
    else:
        ei = betaLoop[i]
        ei1 = betaLoop[0]
            
    turn = findDirection(e.commonNode(ei1),e,ei1)
    
    if prev_turn == 999:
        flag = 1 
        sign = 1
        flags[1] = sign
    elif turn !=0 and turn == prev_turn:
        flag = -1*flag
        sign = -1*flags[flag]
        flags[flag] = sign
    elif turn !=0 and turn != prev_turn:
        flag = -1*flag
        if flag in flags:
            sign = flags[flag]
        else:
            flags[flag] = 1
            sign =1
    else:
        sign = flags[flag]
            
        
    if orientation[e] == 0:
        betaLoopHorizontal.append(e)
    else:
        betaLoopVertical.append(e)
    
    signs2[ei1] = sign
    
    prev_turn = turn
    prev_sign = sign
    
    print(ei1.source().index(),ei1.target().index(), signs2[ei1])


#for e in betaLoop:
 #   print(e.source().index(),e.target().index(),signs2[e])

180.0 -90.0
5 9 1
90.0 0.0
9 10 1
180.0 90.0
6 10 -1
-90.0 0.0
6 7 1
180.0 90.0
3 7 -1
-90.0 0.0
3 5 1


In [20]:
##Setting variable names

alphaVerticalDecisionVariables = {}
betaVerticalDecisionVariables = {}
alphaHorizontalDecisionVariables = {}
betaHorizontalDecisionVariables = {}

for i, e in enumerate(alphaLoopVertical):
    var = LpVariable('eav' + str(i) ,lowBound = 0.1) 
    alphaVerticalDecisionVariables[e] = var
    
    if e in betaLoopVertical:
        betaVerticalDecisionVariables[e] = var
    
for i, e in enumerate(betaLoopVertical):
    var = LpVariable('ebv' + str(i) ,lowBound = 0.1) 
    if e not in alphaLoopVertical:
        betaVerticalDecisionVariables[e] = var

for i, e in enumerate(alphaLoopHorizontal):
    var = LpVariable('eah' + str(i) ,lowBound = 0.1) 
    alphaHorizontalDecisionVariables[e] = var
    
    if e in betaLoopVertical:
        betaHorizontalDecisionVariables[e] = var
    
for i, e in enumerate(betaLoopHorizontal):
    var = LpVariable('ebh' + str(i) ,lowBound = 0.1) 
    if e not in alphaLoopHorizontal:
        betaHorizontalDecisionVariables[e] = var
        

In [21]:
#check if variable names are consistent

print("betaLoopHorz")
for e in betaLoopHorizontal:
    print(e.source().index(),e.target().index(), betaHorizontalDecisionVariables[e])
    
print("alphaLoopHorz")
for e in alphaLoopHorizontal:
    print(e.source().index(),e.target().index(), alphaHorizontalDecisionVariables[e])
print("betaLoopVer")
for e in betaLoopVertical:
    print(e.source().index(),e.target().index(),betaVerticalDecisionVariables[e])
print("alphaLoopVer")
for e in alphaLoopVertical:
    print(e.source().index(),e.target().index(),alphaVerticalDecisionVariables[e])
    

betaLoopHorz
3 5 ebh0
9 10 ebh1
6 7 ebh2
alphaLoopHorz
betaLoopVer
5 9 ebv0
6 10 eav1
3 7 ebv2
alphaLoopVer
10 6 eav0
6 10 eav1


In [23]:
setOfVariables = set(list(betaHorizontalDecisionVariables.values()) + list(alphaHorizontalDecisionVariables.values()) +list(betaVerticalDecisionVariables.values())+list(alphaVerticalDecisionVariables.values()))

model= LpProblem("Network Flow Problem", LpMinimize)

# The objective function
model += lpSum([e for e in setOfVariables]), "Sum of flow across all edges"

# Constraints, if there are more edges going the 'opposite' direction we multiply it by -1 so that a solution exists
if checkCoeff(betaLoopHorizontal, signs2) == 1:  
    model += lpSum([signs2[e]*betaHorizontalDecisionVariables[e] for e in betaHorizontalDecisionVariables.keys() ]) == 1, "Sum of flow across all horizontal edges in beta loop"
    
elif checkCoeff(betaLoopHorizontal, signs2) == -1:  
    model += lpSum([signs2[e]*betaHorizontalDecisionVariables[e] for e in betaHorizontalDecisionVariables.keys() ]) == -1, "Sum of flow across all horizontal edges in beta loop"
    
if checkCoeff(alphaLoopVertical, signs1) == 1:  
    model += lpSum([signs1[e]*alphaVerticalDecisionVariables[e] for e in alphaVerticalDecisionVariables.keys() ]) == 1, "Sum of flow across all vertical edges in alpha loop"
elif checkCoeff(alphaLoopVertical, signs1) == -1:  
    model += lpSum([signs1[e]*alphaVerticalDecisionVariables[e] for e in alphaVerticalDecisionVariables.keys() ]) == 1, "Sum of flow across all vertical edges in alpha loop"

    
model += lpSum([signs2[e]*betaVerticalDecisionVariables[e] for e in betaVerticalDecisionVariables.keys() ]) == 0, "Sum of flow across all vertical edges in beta loop"

model += lpSum([signs1[e]*alphaHorizontalDecisionVariables[e] for e in alphaHorizontalDecisionVariables.keys() ]) == 0, "Sum of flow across all horizontal edges in alpha loop"
    
        
model.solve()
print("Status:", LpStatus[model.status])

# Each of the variables is printed with its resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("The sum of flow = ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/loke/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/6d18470a0a2a45d59deb760864135d4c-pulp.mps branch printingOptions all solution /tmp/6d18470a0a2a45d59deb760864135d4c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 25 RHS
At line 30 BOUNDS
At line 38 ENDATA
Problem MODEL has 4 rows, 7 columns and 8 elements
Coin0008I MODEL read with 0 errors
Presolve 1 (-3) rows, 3 (-4) columns and 3 (-5) elements
0  Obj 1.6 Primal inf 0.699999 (1)
1  Obj 2.3
Optimal - objective value 2.3
After Postsolve, objective 2.3, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2.3 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
eav0 = 0.9
eav1 = 0.1
ebh0 = 0.8
ebh1 = 0.1
ebh2 = 0.1
ebv0 

/home/loke/.local/lib/python3.9/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


betaLoopHorz

3 5 ebh0

9 10 ebh1

6 7 ebh2

alphaLoopHorz :None

betaLoopVer

5 9 ebv0

6 10 eav1

3 7 ebv2

alphaLoopVer

10 6 eav0

6 10 eav1